# Survival Curve Estimation

- How the **Kaplan-Meier** model works and how to fit, visualize, and interpret it
- Apply this model to explore how categorical variables affect survival
- Learn how to supplement your analysis using hypothesis testing methods like the log-rank test

## Kaplan-Meier Estimator

The most widely used method for estimating survival functions

- Also known as the **Product-Limit Estimator**/**K-M Estimator**
- Computes survival probabilities and estimates survival functions
- It is a **Non-parametric method** 
  - Construct survival curve solely from collected data
  - Does not assume the underlying distribution has specific parameter

### The Math Intuition

- The **K-M Estimator** is built on the rule of probability
- Given a duration time $t_i$, we can measure the number of events $d_i$ that happened at $t_i$ and the number of individuals $n_i$ that survived up to $t_i$

$$S(t)=P_{i:t_i\leq{t}}(1-\frac{d_i}{n_i})$$

$\frac{d_i}{n_i}$ is the percent of chance of an event happening at $t_i$

$1 - \frac{d_i}{n_i}$ is the percent of chance of survival at $t_i$

### Why is it called the Product-Limit Estimator?

Suppose we have events at 3 times: 1, 2, 3

The survival rate at $t=2$ is:

$$S(t=2)=(1-\frac{d_1}{n_1}) \cdot (1-\frac{d_2}{n_2})$$

The survival rate at $t=3$ is:

$$S(t=3) = S(t=2) \cdot (1-\frac{d_3}{n_3})$$

This intuitive property is behind the name **Product-Limit Estimator**

- *The Survival Rate at time $t$ is equal to the product of the percentage of chance of surviving at time $t$ and each prior time*

### Assumptions to Keep in Mind

When using K-M Estimator, the data **must satisfy the following conditions**:

- The events are **Unambiguously defined** (happens clearly at a specified time)
- **Survival probabilities are comparable in all subjects** (do not depend on when they entered the study or not)
- **Censorship is non-informative** (censored observations have the same survival prospects as observations that continue)

### Kaplan-Meier Estimator With `lifelines`

In [1]:
# Import the KaplanMeierFitter
from lifelines import KaplanMeierFitter
import pandas as pd

In [2]:
# Our dataset
regimes = pd.read_csv("data/regimes.csv")
display(regimes.head())
display(regimes.shape)

ctryname  cowcode2  politycode un_region_name un_continent_name  \
0  Afghanistan       700       700.0  Southern Asia              Asia   
1  Afghanistan       700       700.0  Southern Asia              Asia   
2  Afghanistan       700       700.0  Southern Asia              Asia   
3  Afghanistan       700       700.0  Southern Asia              Asia   
4  Afghanistan       700       700.0  Southern Asia              Asia   

                   ehead                                     leaderspellreg  \
0    Mohammad Zahir Shah  Mohammad Zahir Shah.Afghanistan.1946.1952.Mona...   
1  Sardar Mohammad Daoud  Sardar Mohammad Daoud.Afghanistan.1953.1962.Ci...   
2    Mohammad Zahir Shah  Mohammad Zahir Shah.Afghanistan.1963.1972.Mona...   
3  Sardar Mohammad Daoud  Sardar Mohammad Daoud.Afghanistan.1973.1977.Ci...   
4    Nur Mohammad Taraki  Nur Mohammad Taraki.Afghanistan.1978.1978.Civi...   

       democracy         regime  start_year  duration  observed  
0  Non-democracy       Monarchy        1946         7         1  
1  Non-democracy  Civilian Dict        1953        10         1  
2  Non-democracy       Monarchy        1963        10         1  
3  Non-democracy  Civilian Dict        1973         5         0  
4  Non-democracy  Civilian Dict        1978         1         0

(1808, 12)

In [3]:
# Instantiate a KaplanMeierFitter class
kmf = KaplanMeierFitter()

In [4]:
# Fit on the data
kmf.fit(
    durations=regimes["duration"], 
    event_observed=regimes["observed"]
)

<lifelines.KaplanMeierFitter:"KM_estimate", fitted with 1808 total observations, 340 right-censored observations>

**The fitted `kmf` model contains valuable information**

In [5]:
# What is the median survival time? (Median lifetime of a regime)
print(kmf.median_survival_time_)

4.0


In [6]:
# What is the probability of survival at each unit of time? (Probability that the regime will keep on living on per year)
print(kmf.survival_function_)

          KM_estimate
timeline             
0.0          1.000000
1.0          0.721792
2.0          0.601973
3.0          0.510929
4.0          0.418835
5.0          0.334008
6.0          0.280902
7.0          0.256825
8.0          0.221975
9.0          0.205147
10.0         0.181350
11.0         0.161200
12.0         0.150332
13.0         0.139197
14.0         0.124896
15.0         0.117845
16.0         0.113521
17.0         0.108010
18.0         0.099792
19.0         0.093632
20.0         0.090994
21.0         0.085479
22.0         0.078473
23.0         0.074113
24.0         0.071088
25.0         0.069543
26.0         0.066231
27.0         0.062745
28.0         0.060900
29.0         0.056971
30.0         0.052751
31.0         0.050353
32.0         0.043160
33.0         0.038364
34.0         0.038364
35.0         0.030143
36.0         0.030143
38.0         0.030143
39.0         0.030143
40.0         0.030143
42.0         0.030143
44.0         0.030143
46.0         0.030143
47.0      

We can predict a future survival probability using the model

In [7]:
# What is the survival probability at a specific time? (Prediction of survival at a specific time)
print(kmf.predict(100)) # What is the probability that a regime will be alive in 100 years

0.015071681623194312


### Benefits and Limitations of K-M Estimator

Benefits | Limitations
:--------|:-----------
Intuitive interpretation of survival probabilities | Survival curve is usually not smooth
Flexible to use on any time-to-event data | If 50%+ of the data is censored, `.median_survival_time_` cannot be calculated (`inf`)
Usually the first model to attempt on time-to-event data | Not effective for analyzing the covariance of survival functions

**Note: Do not remove Censored Data just so you can run K-M**

- Removing censored data from survival analysis will lead to a biased survival function

### Application on `echocardiogram` Data

- **Does pericardial effusion, fluid build-up around the heart, affects heart attack patients' survival outcomes?**
- **Compare survival distributions from patients with and without pericardial effusion**

In [8]:
# Our dataset
echocardiogram = pd.read_csv("data/echocardiogram.csv")
display(echocardiogram.head())
display(echocardiogram.shape)

survival  alive   age  pericardialeffusion  fractionalshortening    epss  \
0      11.0    0.0  71.0                  0.0                 0.260   9.000   
1      19.0    0.0  72.0                  0.0                 0.380   6.000   
2      16.0    0.0  55.0                  0.0                 0.260   4.000   
3      57.0    0.0  60.0                  0.0                 0.253  12.062   
4      19.0    1.0  57.0                  0.0                 0.160  22.000   

    lvdd  wallmotion-score  wallmotion-index   mult  name group  aliveat1  
0  4.600              14.0              1.00  1.000  name     1       0.0  
1  4.100              14.0              1.70  0.588  name     1       0.0  
2  3.420              14.0              1.00  1.000  name     1       0.0  
3  4.603              16.0              1.45  0.788  name     1       0.0  
4  5.750              18.0              2.25  0.571  name     1       0.0

(133, 13)

In [9]:
# Only keep ths needed columns
echocardiogram = echocardiogram[["survival", "alive", "age", "pericardialeffusion", "name"]]
echocardiogram["observed"] = 1 # Assuming no censorship

# Dropping the NA's for now
echocardiogram = echocardiogram.dropna(axis="index")

display(echocardiogram.head())
display(echocardiogram.shape)

survival  alive   age  pericardialeffusion  name  observed
0      11.0    0.0  71.0                  0.0  name         1
1      19.0    0.0  72.0                  0.0  name         1
2      16.0    0.0  55.0                  0.0  name         1
3      57.0    0.0  60.0                  0.0  name         1
4      19.0    1.0  57.0                  0.0  name         1

(125, 6)

In [10]:
# Patients with pericardial effusion
has_pericardial_effusion = echocardiogram[echocardiogram["pericardialeffusion"] == 1]
display(has_pericardial_effusion.head())
display(has_pericardial_effusion.shape)

survival  alive     age  pericardialeffusion  name  observed
11      52.0    0.0  62.000                  1.0  name         1
15      24.0    0.0  55.000                  1.0  name         1
16       0.5    1.0  69.000                  1.0  name         1
17       0.5    1.0  62.529                  1.0  name         1
19       1.0    1.0  66.000                  1.0  name         1

(24, 6)

In [11]:
# Patients without pericardial effusion
none_pericardial_effusion = echocardiogram[echocardiogram["pericardialeffusion"] == 0]
display(none_pericardial_effusion)
display(none_pericardial_effusion.shape)

survival  alive   age  pericardialeffusion  name  observed
0        11.0    0.0  71.0                  0.0  name         1
1        19.0    0.0  72.0                  0.0  name         1
2        16.0    0.0  55.0                  0.0  name         1
3        57.0    0.0  60.0                  0.0  name         1
4        19.0    1.0  57.0                  0.0  name         1
..        ...    ...   ...                  ...   ...       ...
128       7.5    1.0  64.0                  0.0  name         1
129      41.0    0.0  64.0                  0.0  name         1
130      36.0    0.0  69.0                  0.0  name         1
131      22.0    0.0  57.0                  0.0  name         1
132      20.0    0.0  62.0                  0.0  name         1

[101 rows x 6 columns]

(101, 6)

In [12]:
# Instantiate Kaplan Meier object for patients with and without pericardial effusion
kmf_has_pe = KaplanMeierFitter()
kmf_no_pe = KaplanMeierFitter()

# Fit Kaplan Meier estimators to each DataFrame
kmf_has_pe.fit(
    durations=has_pericardial_effusion["survival"], 
    event_observed=has_pericardial_effusion["observed"]
)

<lifelines.KaplanMeierFitter:"KM_estimate", fitted with 24 total observations, 0 right-censored observations>

In [13]:
kmf_no_pe.fit(
    durations=none_pericardial_effusion["survival"], 
    event_observed=none_pericardial_effusion["observed"]
)

<lifelines.KaplanMeierFitter:"KM_estimate", fitted with 101 total observations, 0 right-censored observations>

In [14]:
# Print out the median survival duration of each group
print("The median survival duration (months) of patients with pericardial effusion: ", kmf_has_pe.median_survival_time_)
print("The median survival duration (months) of patients without pericardial effusion: ", kmf_no_pe.median_survival_time_)

The median survival duration (months) of patients with pericardial effusion:  12.0
The median survival duration (months) of patients without pericardial effusion:  25.0


In [15]:
from lifelines.datasets import load_waltons
waltons = load_waltons()
waltons.head()

T  E    group
0   6.0  1  miR-137
1  13.0  1  miR-137
2  13.0  1  miR-137
3  13.0  1  miR-137
4  19.0  1  miR-137